In [ ]:
# Setup Python path to import src modules
import sys
from pathlib import Path

# Add project root to path
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project root added to path: {project_root}")

In [1]:
import pandas as pd
import numpy as np

In [ ]:
# Import preprocessing function
from src.preprocessing import preprocess

In [ ]:
df = pd.read_csv('../data/raw/manhattan.csv')

In [ ]:
df_preprocessed = preprocess(df)

In [ ]:
df_preprocessed.shape

In [ ]:
df_preprocessed.columns

In [ ]:
df_preprocessed.head()

In [ ]:
import joblib
from src.modeling import train_test_split_df, train_linear_regression, train_random_forest, train_xgboost, evaluate_model, model_comparison_table

In [ ]:
X_train, X_test, y_train, y_test = train_test_split_df(df_preprocessed, 'rent')

In [ ]:
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

In [ ]:
model_lr = train_linear_regression(X_train, y_train)

In [ ]:
model_rf = train_random_forest(X_train, y_train)

In [ ]:
model_xgb = train_xgboost(X_train, y_train)

In [ ]:
model_lr

In [ ]:
model_rf

In [ ]:
model_xgb

In [ ]:
results = {}
results["Linear Regression"] = evaluate_model(model_lr, X_test, y_test)
results["Random Forest"] = evaluate_model(model_rf, X_test, y_test)
results["XGBoost"] = evaluate_model(model_xgb, X_test, y_test)

In [ ]:
comparison_df = model_comparison_table(results)

In [ ]:
comparison_df

In [ ]:
joblib.dump(model_xgb, '../outputs/models/best_model.pkl')
'../outputs/models/best_model.pkl'

In [ ]:
import shap

In [ ]:
explainer = shap.TreeExplainer(model_xgb)

In [ ]:
shap_values = explainer.shap_values(X_test)

In [ ]:
shap.summary_plot(shap_values, X_test)

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type='bar')

In [ ]:
X_train.columns.tolist()